# 导入库

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import os
import sys
import time
from tensorflow import keras

print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__ + "的版本为：" + module.__version__)

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
matplotlib的版本为：3.0.3
numpy的版本为：1.16.2
pandas的版本为：0.25.3
sklearn的版本为：0.20.3
tensorflow的版本为：2.0.0
tensorflow_core.keras的版本为：2.2.4-tf


# 导入数据

In [2]:
minst = keras.datasets.mnist

# 数据拆分

In [3]:
img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = minst.load_data()

if keras.backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


# 数据规范化

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255
x_test = x_test / 255

x_train.shape

(60000, 28, 28, 1)

# 独热编码

In [5]:
y_train_onehot = tf.keras.utils.to_categorical(y_train)
y_test_onehot = tf.keras.utils.to_categorical(y_test)

# 搭建卷积神经网络模型

In [6]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               204928    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

# 设置优化器、损失函数

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 模型训练

In [9]:
history = model.fit(x_train, y_train_onehot, batch_size = 256, epochs = 10, verbose=1, validation_data = (x_test, y_test_onehot))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 40s 670us/sample - loss: 0.3792 - accuracy: 0.8845 - val_loss: 0.0750 - val_accuracy: 0.9743
Epoch 2/10
60000/60000 [==============================] - 44s 726us/sample - loss: 0.1136 - accuracy: 0.9659 - val_loss: 0.0492 - val_accuracy: 0.9843
Epoch 3/10
60000/60000 [==============================] - 40s 674us/sample - loss: 0.0827 - accuracy: 0.9759 - val_loss: 0.0410 - val_accuracy: 0.9865
Epoch 4/10
60000/60000 [==============================] - 43s 719us/sample - loss: 0.0671 - accuracy: 0.9801 - val_loss: 0.0346 - val_accuracy: 0.9883
Epoch 5/10
60000/60000 [==============================] - 40s 674us/sample - loss: 0.0566 - accuracy: 0.9837 - val_loss: 0.0333 - val_accuracy: 0.9889
Epoch 6/10
60000/60000 [==============================] - 39s 655us/sample - loss: 0.0493 - accuracy: 0.9847 - val_loss: 0.0303 - val_accuracy: 0.9894
Epoch 7/10
60000/60000 [====================

# 结果检测

In [10]:
score = model.evaluate(x_test, y_test_onehot, verbose=0)

In [11]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02578243754482901
Test accuracy: 0.9918
